In [1]:
import pandas as pd

### Step 0. 청크 생성

In [2]:
# 데이터 준비

welfare_data = pd.read_csv("../data/welfare_dataframe.csv")

In [3]:
# 청크 생성

def make_policy_chunk(row):
    return f"""[정책명] {row['servNm']}

[지원유형] {row['지원유형']}
[지원내용] {row['alwServCn']}

[대상 기준] 
{row['sprtTrgtCn']}

[선정기준] 
{row['slctCritCn']}

[신청안내] {row['신청안내']}
[지원주기] {row['sprtCycNm']}
[시행지역] {row['ctpvNm']} > {row['sggNm']}
[시행기간] {row['시행기간_표기']}

[상세보기] {row['servDtlLink']}
"""

welfare_data["rag_청크"] = welfare_data.apply(make_policy_chunk, axis=1)

### Step 1. 청크 리스트 만들기

In [6]:
documents = welfare_data["rag_청크"].tolist()

### Step 2. Embedding model 선택

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

/var/folders/m1/bqb2x_hs2p3c9xfm8sbxkgcr0000gn/T/ipykernel_66951/813552778.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
/Users/jeon/.pyenv/versions/3.11.10/envs/EverGuide/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

You try to use a model that was created with version 3.3.1, however, your version is 3.1.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

### Step 3. Document 리스트 생성 (passage: prefix 포함!)

In [8]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="passage: " + row["rag_청크"],  # 꼭 prefix 넣기
        metadata={"servId": row["servId"], "servNm": row["servNm"]}  # 추가로 servNm 등 넣어도 좋음
    )
    for _, row in welfare_data.iterrows()
]

### Step 4. FAISS 벡터 저장소에 인덱싱

In [9]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

### Step 5. 검색(예시)

In [22]:
query = "query: 노년층 의료지원 정책이 뭐야?"
results = vectorstore.similarity_search(query, k=3)

print(results)

[Document(metadata={'servId': 'WLF00000339', 'servNm': '노인건강진단 '}, page_content='passage: [정책명] 노인건강진단 \n\n[지원유형] 기타\n[지원내용] 경제적인 어려움으로 의료서비스 혜택에서 소외되기 쉬운 65세 이상 국민기초생활보장대상자 및 차상위자 중 희망자에게 조기치료 서비스를 제공함으로써 건강한 노후생활을 보장\n\n[대상 기준] \n65세 이상 국민기초생활보장수급자 및 차상위 의료급여수급자 중 희망자\n\n[선정기준] \n기초생활보장대상자 및 차상위계층\n\n[신청안내] 신청 방법에 대한 정보가 제공되지 않았습니다.\n[지원주기] 1회성\n[시행지역] 경상남도 > 밀양시\n[시행기간] 무기한\n\n[상세보기] https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId=WLF00000339&wlfareInfoReldBztpCd=02\n'), Document(metadata={'servId': 'WLF00002420', 'servNm': '노인복지 개선 (저소득노인 건강보험 및 장기요양보험료 지원)'}, page_content='passage: [정책명] 노인복지 개선 (저소득노인 건강보험 및 장기요양보험료 지원)\n\n[지원유형] 현물\n[지원내용] 국민건강보험료 및 노인장기요양보험료 지원\n\n[대상 기준] \n차상위계층 주민 중 보험료 부과액이 월 15,000원 이하인 가구 중\r\nㆍ 만65세 이상의 노인세대\r\nㆍ 만65세 미만의 노인성 질병을 가진 사람이 있는 세대\r\nㆍ 그 밖에 구청장이 보험료 지원이 필요하다고 인정하는 저소득가구\n\n[선정기준] \n자산 5400000원 이하\n\n[신청안내] 별도 신청절차 없음\r\n(국민건강보험공단에서 대상자 발송)\n[지원주기] 월\n[시행지역] 서울특별시 > 강동구\n[시행기간] 무기한\n\n[상세보기] https://www.bokji

### Step 6. LLM을 활용한 RAG 질의응답 흐름

In [13]:
from langchain.chains import RetrievalQA
from langchain_community.chat_models import ChatOllama

# LLM 선택: Exaone
llm_EXAONE = ChatOllama(model="exaone-3.0-7.8b-it-Q8_0:latest")

# RAG QA Chain 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_EXAONE,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",  # 또는 "map_reduce", "refine" 등
)

# 질의 테스트
query = "만 70세 이상 노인을 위한 복지 정책이 뭐가 있어?"
result = qa_chain.run(query)

print("🧠 RAG 응답 결과:\n", result)

/var/folders/m1/bqb2x_hs2p3c9xfm8sbxkgcr0000gn/T/ipykernel_66951/1835129823.py:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain.run(query)


🧠 RAG 응답 결과:
 

만 70세 이상 노인을 위한 다양한 복지 정책들이 존재합니다. 여기 몇 가지 예시를 들어볼게요:

1. **노인 목욕 및 이·미용권 지원 (금산군)**:
   - **지원내용**: 1인 연 16매 지원(목욕 및 이미용권)
   - **대상 기준**: 70세 이상 생계·의료급여 수급자
   - **신청안내**: 별도 신청 없이 직접 지원
   - **시행기간**: 무기한
  
2. **70세 이상 어르신 목욕비 및 이미용비 지원 (예산군)**:
   - **지원내용**: 바우처카드 지원(1인당 연 108천원 충전, 반기별 54천원 충전)
   - **대상 기준**: 관내 거주 중인 70세 이상 어르신
   - **신청안내**: 별도 신청 없이 직접 지원
   - **시행기간**: 무기한
  
3. **노인복지관 운영 (화성시)**:
    - **지원내용**: 노인복지관 운영을 통해 다양한 여가 및 복지 서비스 제공
    - **대상 기준**: 화성시에 주소지를 둔 만 60세 이상 노인
    - **신청안내**: 별도 신청 없이 직접 지원
    - **시행기간**: 무기한
  
위의 정책들은 지역별로 다소 차이가 있지만, 공통적으로 만 70세 이상 노인을 대상으로 하고 있습니다. 추가적인 세부 사항이나 최신 정보는 해당 기관의 웹사이트를 통해 확인하시길 권장드립니다.


### Step 6-1. 추천 이유 설명(확장) - Rationale

In [26]:
example_question = "65세 여성 노인이 보건의료 관련해서 제주도에서 지원받을 수 있는 정책은 뭐가 있어?" 
example_query = f'query: {example_question}'

rag_results = vectorstore.similarity_search(example_query, k=3)

In [27]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. 프롬프트 준비
explanation_template = PromptTemplate.from_template("""
사용자가 다음과 같은 질문을 했습니다:
"{question}"

이에 대해 다음과 같은 정책이 추천되었습니다:
"{policy}"

이 정책이 사용자에게 적합한 이유를 알려주세요. 
- 어떤 대상이나 조건이 일치해서 추천되었는지 설명해주세요.
- 노년층도 이해할 수 있도록 친절하게 알려주세요.
""")

# 2. LLMChain 생성
explanation_chain = LLMChain(
    llm=llm_EXAONE,
    prompt=explanation_template,
    verbose=True
)

# 3. 실행
result = explanation_chain.invoke({
    "question": example_query,
    "policy": rag_results
})

print(result["text"])


/var/folders/m1/bqb2x_hs2p3c9xfm8sbxkgcr0000gn/T/ipykernel_66951/1524851847.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  explanation_chain = LLMChain(




> Entering new LLMChain chain...
Prompt after formatting:

사용자가 다음과 같은 질문을 했습니다:
"query: 65세 여성 노인이 보건의료 관련해서 제주도에서 지원받을 수 있는 정책은 뭐가 있어?"

이에 대해 다음과 같은 정책이 추천되었습니다:
"[Document(metadata={'servId': 'WLF00001754', 'servNm': '노인건강진단 '}, page_content='passage: [정책명] 노인건강진단 \n\n[지원유형] 서비스\n[지원내용] 1) 지원기준 \n  - 진료비 : 1인당 (115,300원)\n   \n2) 검진항목 : 개인질환별 정밀검진\n\n[대상 기준] \n1) 지원대상 \r\n - 65세 이상 기초생활수급자 및 차상위 계층 노인 중 건강진단희망자\r\n    (단, 전년도 수검자중 건강한자는 제외)\r\n   \r\n2) 선정기준 \r\n - 1. 2차 노인건강진단 결과 유질환자로 판정된 어르신에 한함.\n\n[선정기준] \n65세 이상 기초의료급여 수급권자 중 건강진단희망자\n\n[신청안내] 1) 신청방법 : 유질환자로 판정된 어르신에 한하여 진료의뢰(시.군- 종합병원)\n2) 검진기관 : 종합병원\n3) 지급시기 및 방법\n - 지급시기 : 유질환자에 대한 진료종료시\n - 지급방법 : 해당시.군에서 진료기관 계좌로 직접 입금\n[지원주기] 1회성\n[시행지역] 제주특별자치도 > 제주시\n[시행기간] 무기한\n\n[상세보기] https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId=WLF00001754&wlfareInfoReldBztpCd=02\n'), Document(metadata={'servId': 'WLF00004304', 'servNm': '노인건강진단사업'}, page_content='passage: [정책명] 노인건강진단사업\n\n[지원유형] 서비스\n[지